In [1]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

CPU times: user 13.5 s, sys: 3.89 s, total: 17.4 s
Wall time: 10.9 s


In [27]:
%%time
# Cấu hình Daskgateway
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
# Khai báo 1 Datacube là dc
dc = datacube.Datacube()

client

An existing cluster was found. Connecting to: easihub.c4dbffb9cdd74f19847caf699e03780c
CPU times: user 30.8 ms, sys: 5.69 ms, total: 36.5 ms
Wall time: 152 ms


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.asia.easi-eo.solutions/services/dask-gateway/clusters/easihub.c4dbffb9cdd74f19847caf699e03780c/status,


In [28]:
# cấu hình thời gian lấy ảnh và tọa độ
date_range = ('2024-05-01', '2024-5-31')
longtitude_range = (105.86575, 105.94120)
latitude_range = (9.65070, 9.69850)

coordinates = (longtitude_range, latitude_range)

In [29]:
# tải dữ liệu vh vv từ vệ tinh sentinel-1
dsvh, dsvv = load_data_sen1(dc, date_range, coordinates)

<xarray.Dataset>
Dimensions:      (time: 2, y: 532, x: 830)
Coordinates:
  * time         (time) datetime64[ns] 2024-05-10T22:46:16.500000 2024-05-22T...
  * y            (y) float64 1.072e+06 1.072e+06 ... 1.067e+06 1.067e+06
  * x            (x) float64 5.95e+05 5.95e+05 5.95e+05 ... 6.033e+05 6.033e+05
    spatial_ref  int32 32648
Data variables:
    vv           (time, y, x) float32 dask.array<chunksize=(1, 532, 830), meta=np.ndarray>
    vh           (time, y, x) float32 dask.array<chunksize=(1, 532, 830), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [30]:
average_vh = calculate_average(dsvh, time_pattern='1M')
progress(average_vh)

VBox()

In [31]:
average_vv = calculate_average(dsvv, time_pattern='1M')
progress(average_vv)

VBox()

In [32]:
%%time
average_vh.compute()
average_vv.compute()

CPU times: user 10.8 ms, sys: 687 µs, total: 11.5 ms
Wall time: 20.5 ms


<xarray.DataArray 'vv' (time: 1, y: 532, x: 830)>
array([[[0.15881254, 0.15881254, 0.19666761, ..., 0.09308143,
         0.09308143, 0.12172551],
        [0.18446743, 0.18446743, 0.22624001, ..., 0.11879794,
         0.11879794, 0.15596746],
        [0.18446743, 0.18446743, 0.22624001, ..., 0.11879794,
         0.11879794, 0.15596746],
        ...,
        [0.12267333, 0.13048397, 0.13048397, ..., 0.2505657 ,
         0.20839426, 0.20839426],
        [0.12267333, 0.13048397, 0.13048397, ..., 0.2505657 ,
         0.20839426, 0.20839426],
        [0.12267333, 0.13048397, 0.13048397, ..., 0.2505657 ,
         0.20839426, 0.20839426]]], dtype=float32)
Coordinates:
  * y            (y) float64 1.072e+06 1.072e+06 ... 1.067e+06 1.067e+06
  * x            (x) float64 5.95e+05 5.95e+05 5.95e+05 ... 6.033e+05 6.033e+05
    spatial_ref  int32 32648
  * time         (time) datetime64[ns] 2024-05-31
Attributes:
    units:         intensity
    nodata:        nan
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [33]:
data_sen2 = load_data_sen2(dc, date_range, coordinates)
notebook_utils.heading(notebook_utils.xarray_object_size(data_sen2))
display(data_sen2)

No datasets require offset correction
The valid_data_mask and scale (no offset) have been applied to the reflectance bands


<xarray.Dataset>
Dimensions:      (time: 6, y: 532, x: 830)
Coordinates:
  * time         (time) datetime64[ns] 2024-05-04T03:35:29.391000 ... 2024-05...
  * y            (y) float64 1.072e+06 1.072e+06 ... 1.067e+06 1.067e+06
  * x            (x) float64 5.95e+05 5.95e+05 5.95e+05 ... 6.033e+05 6.033e+05
    spatial_ref  int32 32648
Data variables:
    red          (time, y, x) float32 dask.array<chunksize=(1, 532, 830), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 532, 830), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 532, 830), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [34]:
%%time
# Tiến hành loại bỏ các vị trí bị mây ảnh hưởng
result = mask_cloud(data_sen2)
progress(result)

{'0': 'no data',
 '1': 'saturated or defective',
 '2': 'dark area pixels',
 '3': 'cloud shadows',
 '4': 'vegetation',
 '5': 'bare soils',
 '6': 'water',
 '7': 'unclassified',
 '8': 'cloud medium probability',
 '9': 'cloud high probability',
 '10': 'thin cirrus',
 '11': 'snow or ice'}

CPU times: user 20.9 ms, sys: 186 µs, total: 21 ms
Wall time: 20 ms


VBox()

In [38]:
# calculate NDVI
dsNDVI = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = dsNDVI["NDVI"]
display(ndvi)

<xarray.DataArray 'NDVI' (time: 6, y: 532, x: 830)>
dask.array<truediv, shape=(6, 532, 830), dtype=float32, chunksize=(1, 532, 830), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2024-05-04T03:35:29.391000 ... 2024-05...
  * y            (y) float64 1.072e+06 1.072e+06 ... 1.067e+06 1.067e+06
  * x            (x) float64 5.95e+05 5.95e+05 5.95e+05 ... 6.033e+05 6.033e+05
    spatial_ref  int32 32648

In [39]:
%%time
# calculate average NDVI
average_ndvi = calculate_average(ndvi,time_pattern="1M")
progress(average_ndvi)

CPU times: user 13.2 ms, sys: 244 µs, total: 13.5 ms
Wall time: 12.6 ms


VBox()

In [40]:
# compute average_ndvi
average_ndvi = average_ndvi.compute()

In [41]:
fill_nan_model = joblib.load('model_train/regressor/500points/fill_nan_model_gb.pkl')

In [42]:
mask = ~np.isnan(average_ndvi)

In [43]:
cloud_free_points = np.stack([average_vh.values[~mask], average_vv.values[~mask]], axis=1)

In [44]:
# X_pred = np.stack([average_vh.values[~mask], average_vv.values[~mask]], axis=1)
average_ndvi.values[~mask] = fill_nan_model.predict(cloud_free_points)
average_ndvi_filled = xr.DataArray(average_ndvi, dims=average_ndvi.dims)

In [45]:
average_ndvi_filled.values

array([[[0.46482947, 0.46482947, 0.55715775, ..., 0.13884138,
         0.13884138, 0.23519422],
        [0.53762186, 0.53762186, 0.54759234, ..., 0.32184023,
         0.32184023, 0.50129557],
        [0.53762186, 0.53762186, 0.54759234, ..., 0.32184023,
         0.32184023, 0.50129557],
        ...,
        [0.31200743, 0.3240093 , 0.3548983 , ..., 0.65298665,
         0.6338502 , 0.6338502 ],
        [0.27736267, 0.26660758, 0.29234853, ..., 0.65298665,
         0.6338502 , 0.6338502 ],
        [0.17999999, 0.18418844, 0.21063744, ..., 0.65298665,
         0.6338502 , 0.6338502 ]]], dtype=float32)

###########################################################################################

In [46]:
print(average_ndvi.shape, average_vh.shape, average_vv.shape)

(1, 532, 830) (1, 532, 830) (1, 532, 830)


In [47]:
def extract_data_point(train, average_ndvi, dsvh, dsvv):
    loaded_datasets = {}
    for idx, point in train.iterrows():
        key = f"point_{idx + 1}"
        try:
            ndvi_data = average_ndvi.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            vh_data = dsvh.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            vv_data = dsvv.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            loaded_datasets[key] = {
                "data": np.stack((ndvi_data, vh_data, vv_data), axis=1),
                "label": point.Code
            }
        except Exception as e:
            # Handle the exception if necessary
            print(f"Error at point {key}: {e}")
    return loaded_datasets

In [48]:
def create_dataset(datasets):
    data_points = []
    labels = []
    for point_key, point_data in datasets.items():
        data_array = point_data['data']
        label = point_data['label']
        for row in data_array:
            data_points.append(row)
            labels.append(label)
    return data_points, labels

In [2]:
geo_path = "train/300point_data/ThuanHoa_DKS_300diem.shp"
# train_path = "train/updated_data/ThuanHoa_DKS_Kappa.shp"
points_label = load_data_geo(geo_path)
points_label

,STT,LULC,Code,geometry
0,1,Dat xay dung,8,POINT Z (601020.539 1068835.423 0.000)
1,2,CLN,5,POINT Z (600992.765 1068892.073 0.000)
2,3,Dat xay dung,8,POINT Z (600983.261 1068711.965 0.000)
3,4,Lua,3,POINT Z (600885.708 1069333.342 0.000)
4,5,Dat xay dung,8,POINT Z (600129.865 1069351.996 0.000)
5,6,CLN,5,POINT Z (600810.711 1068891.221 0.000)
6,7,CLN,5,POINT Z (600786.149 1069268.608 0.000)
7,8,Lua,3,POINT Z (600604.436 1069084.169 0.000)
8,9,CLN,5,POINT Z (600517.093 1069186.396 0.000)
9,10,CLN,5,POINT Z (600990.227 1068837.253 0.000)


In [50]:
data_points = extract_data_point(points_label, average_ndvi_filled, average_vh, average_vv)
X_train, y_train = create_dataset(data_points)

In [72]:
loaded_model = joblib.load(os.path.join("model_train/new_classify", "classify_knn.joblib"))

In [73]:
%time
# dự đoán
y_pred = loaded_model.predict(X_train)


CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 13.8 µs


In [74]:
acc = accuracy_score(y_pred, y_train)
acc

0.2842809364548495

In [71]:
y_pred

array([5, 8, 5, 3, 8, 5, 3, 5, 3, 5, 3, 3, 5, 3, 5, 8, 5, 8, 5, 8, 9, 8,
       3, 9, 8, 5, 5, 9, 5, 5, 5, 9, 8, 9, 3, 8, 3, 5, 5, 5, 5, 3, 3, 3,
       8, 5, 8, 5, 9, 8, 3, 5, 5, 9, 5, 5, 8, 3, 5, 5, 9, 3, 5, 9, 9, 3,
       3, 8, 5, 3, 5, 9, 5, 5, 3, 3, 5, 5, 3, 3, 5, 9, 9, 3, 5, 5, 5, 5,
       5, 3, 3, 5, 3, 3, 3, 3, 3, 5, 5, 9, 3, 5, 3, 9, 5, 9, 5, 9, 5, 5,
       5, 5, 3, 9, 9, 3, 3, 3, 5, 5, 5, 5, 9, 9, 5, 8, 5, 9, 3, 3, 5, 3,
       5, 3, 3, 5, 3, 3, 3, 5, 5, 3, 3, 3, 5, 5, 5, 5, 5, 5, 9, 9, 5, 5,
       9, 5, 5, 5, 5, 5, 3, 8, 9, 5, 3, 3, 3, 8, 5, 3, 3, 3, 8, 3, 5, 3,
       3, 3, 3, 3, 3, 3, 5, 3, 5, 3, 5, 5, 5, 5, 3, 3, 5, 3, 3, 9, 3, 3,
       5, 5, 5, 5, 6, 9, 9, 5, 9, 5, 9, 8, 5, 9, 3, 9, 8, 9, 9, 9, 5, 5,
       5, 9, 9, 9, 5, 5, 3, 8, 3, 5, 5, 5, 8, 5, 5, 8, 5, 5, 8, 3, 5, 8,
       5, 5, 3, 8, 5, 5, 3, 8, 5, 8, 9, 5, 3, 3, 5, 5, 3, 9, 9, 8, 9, 5,
       5, 5, 3, 5, 3, 3, 8, 3, 5, 5, 9, 5, 3, 3, 5, 8, 5, 5, 5, 5, 5, 9,
       5, 5, 3, 5, 5, 5, 9, 3, 3, 3, 5, 3, 3])

In [ ]:
y_train